In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Load pre-trained CNN model
model = tf.keras.applications.VGG16(weights='imagenet', include_top=False)

# Get the first convolutional layer
conv_layer = model.get_layer('block1_conv1')

# Pick a filter to visualize
filter_index = 0

# Generate an input image that activates the filter
input_image = np.random.random((1, 224, 224, 3))
while True:
    activation = filter_activation(input_image, conv_layer, filter_index)
    if activation.mean() > 0.5:
        break
    input_image = np.random.random((1, 224, 224, 3))

# Define a function to calculate the output of a layer given its input
def layer_output(input_image, layer):
    # Create a new model that outputs the output of the target layer given its input
    new_model = tf.keras.models.Model(inputs=model.inputs, outputs=layer.output)
    # Calculate the output of the layer
    output = new_model.predict(input_image)
    return output

# Define a function to reverse the convolutional operation
def deconvolve(output, weights):
    # Transpose the weights and flip them along the spatial dimensions
    flipped_weights = np.transpose(weights, (0, 1, 3, 2))
    flipped_weights = np.flip(flipped_weights, axis=1)
    flipped_weights = np.flip(flipped_weights, axis=2)
    # Pad the output with zeros to match the shape of the deconvolved output
    output_shape = (1,) + tuple(flipped_weights.shape[:3])
    output_padded = np.zeros(output_shape)
    output_padded[:, :output.shape[1], :output.shape[2], :] = output
    # Perform the deconvolution
    deconvolved_output = tf.nn.conv2d_transpose(output_padded, flipped_weights, output_shape, (1, 1), 'SAME')
    return deconvolved_output.numpy()

# Define a function to visualize the filters in a layer
def visualize_filters(layer):
    # Get the weights of the layer
    weights = layer.get_weights()[0]
    # Create a figure to display the filters
    fig, axs = plt.subplots(nrows=8, ncols=8, figsize=(10, 10))
    # Loop over the filters in the layer
    for i in range(weights.shape[3]):
        # Generate an image that activates the filter
        input_image = np.random.random((1, 224, 224, 3))
        while True:
            activation = filter_activation(input_image, layer, i)
            if activation.mean() > 0.5:
                break
            input_image = np.random.random((1, 224, 224, 3))
        # Reverse the convolutional operation to generate an image that matches the activation of the filter
        output = layer_output(input_image, layer)
        deconvolved_output = deconvolve(output, weights[:, :, :, i:i+1])
        # Rescale the image to [0, 1] range and display it
        axs[i // 8, i % 8].imshow(deconvolved_output[0, :, :, 0], cmap='gray')
        axs[i // 8, i % 8].axis('off')
    plt.show()

# Visualize the filters in the first convolutional layer
visualize_filters(conv_layer)